In [1]:
import urllib
import json
import requests
import pprint

# For XML -> json
import xml.etree.ElementTree as ET
from xmljson import badgerfish as bf

In [2]:
# For url -> json
!curl -i -X PUT -H "Content-Type: application/json" -d

curl: option -d: requires parameter
curl: try 'curl --help' or 'curl --manual' for more information


Sign up zws_id here
https://www.zillow.com/howto/api/APIOverview.htm?utm_source=email&utm_medium=email&utm_campaign=emo-apiregistration-api

In [3]:
# Put in your zws_id here as a string
zws_id="XXX"

In [4]:
# Use redirected request of zpid only url to get address on zillow
import requests
response = requests.get("https://www.zillow.com/homedetails/16842323_zpid/")
if response.history:
    print ("Request was redirected")
    for resp in response.history:
        print (resp.status_code, resp.url)
    print ("Final destination:")
    print (response.status_code, response.url)
else:
    print ("Request was not redirected")

Request was redirected
301 https://www.zillow.com/homedetails/16842323_zpid/
Final destination:
200 https://www.zillow.com/homedetails/4215-Porte-De-Palmas-UNIT-203-San-Diego-CA-92122/16842323_zpid/


In [18]:
def API_URL(zws_id, zpid):
    response = requests.get("https://www.zillow.com/homedetails/" + str(zpid) + "_zpid/")
    address = response.url.split('/')[-3]
    url = "http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=" + zws_id + "&address="
    street = address.split('-')[:-4]
    for i in range(len(street)-1):
        url += street[i]
        url += "+"
    url += street[-1] + "&citystatezip=" + address.split('-')[-4] + "+" + address.split('-')[-3]
    url += "%2C+CA&rentzestimate=true"
    return url


def rentData_URL(zpid):
    return "https://www.zillow.com/ajax/homedetail/HomeValueChartData.htm?mt=9&zpid=" + str(zpid) + "&format=json"

result_str = '{http://www.zillow.com/static/xsd/SearchResults.xsd}searchresults'

historical rent zestimate values reference

https://www.zillow.com/advice-thread/Can-I-get-historical-zestimate-values-from-the-API/471215/

https://www.zillow.com/ajax/homedetail/HomeValueChartData.htm?mt=9&zpid=16842322&format=json
https://www.zillow.com/ajax/homedetail/HomeValueChartData.htm?mt=9&zpid=16842322

The result json will contain only this home value.

In [20]:
# 'a' for appending newly scraping data
outfile = open('data.json', 'a')

merged_json = dict()
for zpid in range(16842450,(16842450+300)):
    json_property = json.loads(json.dumps(bf.data(ET.fromstring(urllib.request.urlopen(API_URL(zws_id,zpid)).read()))))
    if json_property[result_str]['message']['code']['$'] != 0:
        continue
    json_propertyResponse = json_property[result_str]['response']
    json_rentHistory = json.loads(urllib.request.urlopen(rentData_URL(zpid)).read())[0]
    json_propertyResponse["HomeValueChartData"] = json.dumps(json_rentHistory)
    merged_json['zpid'] = zpid
    merged_json['data'] = json_propertyResponse
    json.dump(merged_json, outfile)
    outfile.write('\n')
outfile.close()
print("Done")

Done


In [21]:
# Load the result json file to dest_json
dest_json = dict()
for line in open('data.json','r'):
    temp = json.loads(line)
    dest_json[temp['zpid']] = temp['data']


In [22]:
# Print one result
pprint.pprint(dest_json[16842300])

{'HomeValueChartData': '{"points": [{"x": 1285830000000, "y": 2300.0}, {"x": '
                       '1288508400000, "y": 2395.0}, {"x": 1291104000000, "y": '
                       '2200.0}, {"x": 1293782400000, "y": 2200.0}, {"x": '
                       '1296460800000, "y": 2175.0}, {"x": 1298880000000, "y": '
                       '2195.0}, {"x": 1301554800000, "y": 2150.0}, {"x": '
                       '1304146800000, "y": 2245.0}, {"x": 1306825200000, "y": '
                       '2250.0}, {"x": 1309417200000, "y": 2200.0}, {"x": '
                       '1312095600000, "y": 2195.0}, {"x": 1314774000000, "y": '
                       '2200.0}, {"x": 1317366000000, "y": 2250.0}, {"x": '
                       '1320044400000, "y": 2200.0}, {"x": 1322640000000, "y": '
                       '2200.0}, {"x": 1325318400000, "y": 2200.0}, {"x": '
                       '1327996800000, "y": 2250.0}, {"x": 1330502400000, "y": '
                       '2250.0}, {"x": 1333177200000, "